# Xception 모델 & Openlogo 데이터셋 학습 

In [64]:
import keras 
from keras.applications import xception
from keras import layers
from keras import preprocessing

base_model = keras.applications.xception.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

norm_layer = keras.layers.BatchNormalization()
x = norm_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
outputs = keras.layers.Dense(2)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_64 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_12  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_12 (Dropout)        (None, 2048)              0         
                                                                 
 dense_12 (Dense)            (None, 2)                 409

In [65]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.6,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/train', 
    target_size=(150, 150),
    batch_size=100, 
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_split/test',
    target_size=(150,150),
    batch_size=100,
    class_mode='binary'
)

Found 18752 images belonging to 2 classes.
Found 8331 images belonging to 2 classes.


In [66]:
# 가장 기본적인 3가지 콜백 
callbacks_list = [

    #과대적합 발생 시 조기 스톱 콜백
    keras.callbacks.EarlyStopping(
        monitor='binary_accuracy', # 모델 검증 손실 모니터링
        patience= 10), # 20에폭동안 과대적합 관찰
    
    # 최고의 가중치 자동 저장 콜백
    keras.callbacks.ModelCheckpoint(
        '/Users/kibeomkim/Desktop/models_saved/my_model.h5', 
        monitor = 'val_loss', 
        save_best_only = True), 

    # 학습률 자동 조정 콜백
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5
    ),

    # 텐서보드 
    keras.callbacks.TensorBoard(
        log_dir = '/Users/kibeomkim/Desktop/my_log_dir',
        histogram_freq = 1, # 1에포크마다 층 출력의 히스토그램 기록
        embeddings_freq = 1
    )
]

In [67]:
import keras.optimizers

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(), 
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 187, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=83, 
    callbacks=callbacks_list 
)

Epoch 1/5


2022-02-21 19:05:07.853892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - ETA: 0s - loss: 0.0581 - binary_accuracy: 0.9950

2022-02-21 19:10:56.907830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - 415s 2s/step - loss: 0.0581 - binary_accuracy: 0.9950 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 0.0010
Epoch 2/5
187/187 [==============================] - 400s 2s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 0.0010
Epoch 3/5
187/187 [==============================] - 393s 2s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 0.0010
Epoch 4/5
187/187 [==============================] - 400s 2s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 0.0010
Epoch 5/5
187/187 [==============================] - 394s 2s/step - loss: 0.0380 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 0.0010


In [59]:
# 모델 저장 
model.save('/Users/kibeomkim/Desktop/models_saved/samsung_model.h5')

# from tensorflow.keras.models import load_model 
# model = load_model('/Users/kibeomkim/Desktop/models_saved/samsung_model.h5)

In [68]:
# 미세조정 
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(learning_rate=1e-5), 
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 187, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=83, 
    callbacks=callbacks_list 
)

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_64 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_12  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_12 (Dropout)        (None, 2048)              0         
                                                                 
 dense_12 (Dense)            (None, 2)                 409

2022-02-21 20:38:39.340467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - ETA: 0s - loss: 0.0389 - binary_accuracy: 0.9975

2022-02-21 20:48:57.577862: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


187/187 [==============================] - 681s 4s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 1.0000e-05
Epoch 2/5
187/187 [==============================] - 683s 4s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 1.0000e-05
Epoch 3/5
187/187 [==============================] - 713s 4s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 1.0000e-05
Epoch 4/5
187/187 [==============================] - 615s 3s/step - loss: 0.0389 - binary_accuracy: 0.9975 - val_loss: 0.0353 - val_binary_accuracy: 0.9977 - lr: 1.0000e-05
Epoch 5/5
187/187 [==============================] - 670s 4s/step - loss: 0.0364 - binary_accuracy: 0.9976 - val_loss: 0.0335 - val_binary_accuracy: 0.9978 - lr: 1.0000e-05


In [69]:
# 모델 저장 
model.save('/Users/kibeomkim/Desktop/models_saved/samsung_model_improved.h5')

# from tensorflow.keras.models import load_model 
# model = load_model('/Users/kibeomkim/Desktop/models_saved/samsung_model.h5)